In [35]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

def scrape_google_flights_all_prices(origin, destination, depart_date):
    # Chrome 옵션 설정
    options = Options()
    options.add_argument("--headless")  # 브라우저 창 없이 실행
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    )  # User-Agent 설정

    # Chrome WebDriver 실행
    driver = webdriver.Chrome(options=options)

    # Google Flights URL 생성
    url = f"https://www.google.com/travel/flights?q=Flights%20from%20{origin}%20to%20{destination}%20on%20{depart_date}"
    driver.get(url)
    
    # 페이지 로딩 대기
    time.sleep(60)  # 동적 콘텐츠 로딩 시간 필요

    try:
        # 전체 가격 정보 추출 (# 인기항공권으로 한정 )
        price_elements = driver.find_elements(By.XPATH, '//div[@jsname="IWWDBc"]//div[@class="U3gSDe"]//div[@class="BVAVmf I11szd POX3ye"]//div[contains(@class,"YMlIz FpEdX")]')
        prices = [price.text.replace("₩","").replace(",","") for price in price_elements]

        # 항공사 정보 추출 (# 인기항공권으로 한정 )
        airline_elements = driver.find_elements(By.XPATH, '//div[@jsname="IWWDBc"]//div[@class="OgQvJf nKlB3b"]//div[@class="KhL0De"]//div[@class="Ir0Voe"]//div[@class="sSHqwe tPgKwe ogfYpf"]')
        airlines = [airline.text.replace("\n"," ").replace("함께 예약된 다구간 항공권 ","") for airline in airline_elements]

        # 가격과 항공사 정보 필터링 (빈 값 제거)
        prices = filter(None, prices)
        airlines = filter(None, airlines)

        # 가격과 항공사 정보 매핑
        results = list(zip(airlines, prices))

        # 출발지, 목적지, 날짜를 포함한 데이터프레임 생성
        df = pd.DataFrame(results, columns=["Airline", "Price"])
        df["Origin"] = origin
        df["Destination"] = destination
        df["Date"] = depart_date

        # 열 순서 재정렬 (Date를 첫 번째 열로 이동)
        df = df[["Date", "Origin", "Destination", "Airline", "Price"]]
        
        # 데이터프레임 출력
        print(f"{depart_date} | {origin} → {destination}")
        print(df)

        # 데이터프레임을 CSV 파일로 저장
        output_file = f"flights_{depart_date}.csv"
        df.to_csv(output_file, index=False, encoding="utf-8")
    except Exception as e:
        print(f"{depart_date} | Failed to extract prices: {e}")
    finally:
        driver.quit()

# 예시 실행
scrape_google_flights_all_prices("ICN", "LHR", "2025-04-11")

2025-04-11 | ICN → LHR
         Date Origin Destination        Airline    Price
0  2025-04-11    ICN         LHR          에어프랑스  1013200
1  2025-04-11    ICN         LHR            KLM  1096700
2  2025-04-11    ICN         LHR          에어차이나  1139093
3  2025-04-11    ICN         LHR         아시아나항공  1398100
4  2025-04-11    ICN         LHR  대한항공버진 애틀랜틱항공  1400900
